# OAS -> Tools
> This notebook is to validate OAS -> Tools (Schema for Function-Calling) conversion and implement python module for that.

In detail, https://chatgpt.com/share/6720630a-827c-8002-85f7-6f02eff3d382

In [ ]:
#| default_exp oas_to_tools

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import json
import openai
import requests

In [ ]:
#| export

def fetch_OAS(url: str, # URL for OAS
              out='openapi.json' # Output path
              ):
    """Fetch OAS from a given URL and save to `.json` file (default = 'openapi.json')"""
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        openapi_json = response.json()
        # Step 2: Save the JSON to a file
        with open('openapi.json', 'w') as f:
            json.dump(openapi_json, f)

        # Load the OpenAPI JSON file as a string
        with open('openapi.json', 'r') as file:
            openapi_data_str = file.read()

        # Replace all instances of "from" with "from_date" and "to" with "to_date"
        modified_data_str = openapi_data_str.replace(': "from"', ': "from_date"').replace(': "to"', ': "to_date"')

        with open(out, 'w') as file:
            file.write(modified_data_str)

        return out

    else:
        print("Failed to fetch OpenAPI JSON:", response.status_code)

In [ ]:
show_doc(fetch_OAS)

---

### fetch_OAS

>      fetch_OAS (url:str, out='openapi.json')

*Fetch OAS from a given URL and save to `.json` file (default = 'openapi.json')*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| url | str |  | URL for OAS |
| out | str | openapi.json | Output path |

In [ ]:
#| export

def parse_oas(oas_file: str # OAS path
              ) -> list: # List of functions information
    """Parse information from OAS to extract information for function generation"""
    # Load the OpenAPI specification
    with open(oas_file, 'r') as file:
        spec = json.load(file)
    
    functions = []
    
    # Iterate through each path in the OpenAPI spec
    for path, methods in spec['paths'].items():
        for method, details in methods.items():
            func_name = f"{method}{path.replace('/', '_').replace('{', '').replace('}', '').replace('-', '_').replace('.', '_')}"
            summary = details.get('summary')
            parameters = details.get('parameters', [])
            
            # Separate path and query parameters
            path_params = [param for param in parameters if param['in'] == 'path']
            query_params = [param for param in parameters if param['in'] == 'query']
            
            # Construct function definition
            func = {
                'name': func_name,
                'description': summary, 
                'path': path,
                'method': method,
                'path_params': path_params,
                'query_params': query_params
            }
            functions.append(func)
    
    return functions

In [ ]:
show_doc(parse_oas)

---

### parse_oas

>      parse_oas (oas_file:str)

*Parse information from OAS to extract information for function generation*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| oas_file | str | OAS path |
| **Returns** | **list** | **List of functions information** |

In [ ]:
#| export

def generate_function_code(func: dict, # Function information
                           base_url: str # API URL
                           ) -> str: # Function code in text format
    """Generate functions corresponding to information given that can send requests to API"""
    # Extract function details
    path = func['path']
    func_name = func['name']
    summary = func['description']
    
    # Define required path parameters (without defaults) and optional query parameters
    path_args = ', '.join([param['name'] for param in func['path_params']])
    optional_query_args = ', '.join([f"{param['name']}=None" for param in func['query_params']])

    # Combine all arguments for the function signature
    all_args = ', '.join(filter(None, [path_args, optional_query_args]))

    # Path and query parameter dictionaries for build_url function
    path_params_dict = "{" + ", ".join([f"'{param['name']}': {param['name']}" for param in func['path_params']]) + "}" if func['path_params'] else "None"
    query_params_dict = "{**{k: v for k, v in locals().items() if v is not None and k in [" + \
                        ", ".join([f"'{param['name']}'" for param in func['query_params']]) + "]}}"
    
    # Currently handling only GET requests
    code = f"""
def {func_name}({all_args}):
    \"\"\"
    {summary}
    \"\"\"
    # Build URL with path and query params
    url = build_url("{base_url}{path}", path_params={path_params_dict}, query_params={query_params_dict})
    response = send_request(url)
    return response
"""
    return code

In [ ]:
show_doc(generate_function_code)

---

### generate_function_code

>      generate_function_code (func:dict, base_url:str)

*Generate functions corresponding to information given that can send requests to API*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| func | dict | Function information |
| base_url | str | API URL |
| **Returns** | **str** | **Function code in text format** |

In [ ]:
#| export

def build_url(path: str, # URL with path
              path_params: dict=None, # Parameters in path
              query_params: dict=None # Parameters in query
              ) -> str: # Complete URL with parameters
    """Build an URL with parameters"""
    # Replace placeholders in the path with actual parameter values
    if path_params:
        for param, value in path_params.items():
            path = path.replace(f"{{{param}}}", str(value))
    
    # Construct query string only from non-None query parameters
    if query_params:
        query_string = []
        for key, value in query_params.items():
            if isinstance(value, list):
                # If the parameter is a list, append each item individually
                for item in value:
                    query_string.append(f"{key}={item}")
            elif value is not None:
                query_string.append(f"{key}={value}")
        
        if query_string:
            path += '?' + '&'.join(query_string)
    
    return path

In [ ]:
#| hide

assert build_url("test/{id}", path_params={'id': 5}, query_params={'now': False}) == "test/5?now=False"

In [ ]:
show_doc(build_url)

---

### build_url

>      build_url (path:str, path_params:dict=None, query_params:dict=None)

*Build an URL with parameters*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| path | str |  | URL with path |
| path_params | dict | None | Parameters in path |
| query_params | dict | None | Parameters in query |
| **Returns** | **str** |  | **Complete URL with parameters** |

In [ ]:
#| export

def send_request(url: str, # URL path
                 method: str='GET', # Method 
                 headers:dict=None, # Header information 
                 data: dict=None # Data for POST request
                 ):
    """Send requests to the specified URL"""

    # Default headers if none provided
    if headers is None:
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    
    # Send the HTTP request based on the method
    if method.upper() == 'GET':
        response = requests.get(url, headers=headers)
    elif method.upper() == 'POST':
        response = requests.post(url, headers=headers, json=data)
    elif method.upper() == 'PUT':
        response = requests.put(url, headers=headers, json=data)
    elif method.upper() == 'DELETE':
        response = requests.delete(url, headers=headers)
    else:
        raise ValueError(f"Unsupported HTTP method: {method}")
    
    # Raise an exception for HTTP error codes (4xx, 5xx)
    response.raise_for_status()
    
    try:
        return response.json()  # Attempt to parse the response as JSON
    except ValueError:
        return response.text

In [ ]:
show_doc(send_request)

---

### send_request

>      send_request (url:str, method:str='GET', headers:dict=None,
>                    data:dict=None)

*Send requests to the specified URL*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| url | str |  | URL path |
| method | str | GET | Method |
| headers | dict | None | Header information |
| data | dict | None | Data for POST request |

In [ ]:
#| export

def generate_functions(oas_file: str, # OAS file path 
                       base_url: str # API URL
                       ) -> dict: # Dictionary of generated functions corresponding to OAS
    """Generate actual functions from the code in text format"""

    functions = parse_oas(oas_file)
    func_list = []
    for func in functions:
        func_list.append(generate_function_code(func, base_url))
    globals_dict = {
        'build_url': build_url,
        'send_request': send_request,
    }
    generated_functions = {}

    for code in func_list:
        exec(code, globals_dict, generated_functions)

    return generated_functions

In [ ]:
show_doc(generate_functions)

---

### generate_functions

>      generate_functions (oas_file:str, base_url:str)

*Generate actual functions from the code in text format*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| oas_file | str | OAS file path |
| base_url | str | API URL |
| **Returns** | **dict** | **Dictionary of generated functions corresponding to OAS** |

In [ ]:
#| export

# Function to generate a tool schema from OpenAPI
def generate_tool_schema(path: str, # Specific API path
                         method: str, # Method of request 
                         operation: dict, # API information 
                         components: dict # Custom types
                         ) -> dict: # Tool schema for FC
    """Generate the schema for a function"""
    
    # Recursively extract the components information
    def resolve_reference(ref, components):
        # Remove the "#/components/schemas/" part of the $ref
        ref_name = ref.split('/')[-1]
        component = components.get(ref_name, {})
        
        if not component:
            raise ValueError(f"Component '{ref_name}' not found in the components schemas.")
        
        # If the component itself contains $ref, resolve it recursively
        if '$ref' in component:
            return resolve_reference(component['$ref'], components)
        
        return component

    description = operation.get('description') if 'description' in operation else operation.get('summary', 'No description')
    tool_schema = {
        "type": "function",
        "function": {
            "name": f"{method}{path.replace('/', '_').replace('{', '').replace('}', '').replace('-', '_').replace('.', '_')}",  # Use operationId or path as name
            "description": description,
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }

    # Extract parameters information
    for param in operation.get('parameters', []):
        param_info = {
            "type": "string",  # Default type
            "description": param.get('description', 'No description')
        }

        # Check for type in schema and set type (array, object, string, etc.)
        if 'schema' in param:
            param_schema = param['schema']
            
            # Handle direct type assignment (string, integer, etc.)
            if 'type' in param_schema:
                param_info['type'] = param_schema['type']

            # If it's an object, resolve it recursively
            if param_info['type'] == 'object' and '$ref' in param_schema:
                referenced_component = resolve_reference(param_schema['$ref'], components)
                param_info.update({
                    "properties": referenced_component.get('properties', {}),
                    "required": referenced_component.get('required', [])
                })

            # Handle arrays and items (especially for $ref and enums)
            elif param_info['type'] == 'array' and 'items' in param_schema:
                items_info = param_schema['items']
                if '$ref' in items_info:
                    ref_name = items_info['$ref'].split('/')[-1]
                    # Resolve the $ref and get the enum values if available
                    referenced_item = resolve_reference(items_info['$ref'], components)
                    param_info['items'] = {"type": "string", "enum": referenced_item.get('enum', [])}
                    param_info['default'] = referenced_item.get('default', None)
                else:
                    param_info['items'] = items_info

            # Handle enums directly within the parameter schema
            elif 'enum' in param_schema:
                param_info['enum'] = param_schema['enum']
                param_info['default'] = param_schema.get('default', None)  # Default value from schema

            # Handle format (e.g., date-time, int32, etc.)
            if 'format' in param_schema:
                param_info['format'] = param_schema['format']

        # Add to parameters properties
        tool_schema["function"]["parameters"]["properties"][param['name']] = param_info
        
        # Mark required parameters
        if param.get('required', False):
            tool_schema["function"]["parameters"]["required"].append(param['name'])

    return tool_schema

In [ ]:
show_doc(generate_tool_schema)

---

### generate_tool_schema

>      generate_tool_schema (path:str, method:str, operation:dict,
>                            components:dict)

*Generate the schema for a function*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| path | str | Specific API path |
| method | str | Method of request |
| operation | dict | API information |
| components | dict | Custom types |
| **Returns** | **dict** | **Tool schema for FC** |

In [ ]:
#| export

def generate_tools(oas_file: str # OAS file path 
                   ) -> list: # Tool schema for Function Calling
    """Generate compatible `tools` for OpenAI Function Calling"""

    with open(oas_file, 'r') as file:
        spec = json.load(file)

    tool_schemas = []
    components = spec.get('components', {}).get('schemas', {})

    for path, methods in spec['paths'].items():
        for method, operation in methods.items():
            tool_schema = generate_tool_schema(path, method, operation, components)
            tool_schemas.append(tool_schema)

    return tool_schemas

In [ ]:
show_doc(generate_tools)

---

### generate_tools

>      generate_tools (oas_file:str)

*Generate compatible `tools` for OpenAI Function Calling*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| oas_file | str | OAS file path |
| **Returns** | **list** | **Tool schema for Function Calling** |

# Example usage of OAS -> Tools

In [ ]:
#| eval: false
api_url = "https://tie.digitraffic.fi"
oas_url = "https://tie.digitraffic.fi/swagger/openapi.json"
oas = fetch_OAS(oas_url)
generated_functions = generate_functions(oas, api_url)
tools = generate_tools(oas)

In [ ]:
#| eval: false

messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, can you show me the latest information about road maintenance tracking route?"}
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [ ]:
#| eval: false

func_name = response.choices[0].message.tool_calls[-1].function.name
args = json.loads(response.choices[0].message.tool_calls[-1].function.arguments)
generated_functions[func_name](**args)

{'type': 'FeatureCollection',
 'dataUpdatedTime': '2024-11-07T10:08:14Z',
 'features': [{'type': 'Feature',
   'properties': {'id': 118710728,
    'time': '2024-11-07T09:10:21Z',
    'created': '2024-11-07T09:12:51Z',
    'tasks': ['OTHER_OPERATIONS_OF_LIGHTING_CONTRACTS'],
    'direction': 158.0,
    'domain': 'state-roads',
    'source': 'Harja/Väylävirasto'},
   'geometry': {'type': 'Point', 'coordinates': [24.654661, 60.916626, 0.0]}},
  {'type': 'Feature',
   'properties': {'id': 118710732,
    'time': '2024-11-07T09:10:30Z',
    'created': '2024-11-07T09:12:51Z',
    'tasks': ['CLEANSING_OF_TRAFFIC_SIGNS'],
    'direction': None,
    'domain': 'state-roads',
    'source': 'Harja/Väylävirasto'},
   'geometry': {'type': 'Point', 'coordinates': [25.494152, 60.983087, 0.0]}},
  {'type': 'Feature',
   'properties': {'id': 118710801,
    'time': '2024-11-07T09:11:31Z',
    'created': '2024-11-07T09:14:51Z',
    'tasks': ['ROAD_STATE_CHECKING'],
    'direction': 0.0,
    'domain': 'stat

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()